In [16]:
#
# hw4pr3titanic:  titanic-passenger clasification via nearest neighbors
#

In [18]:
# libraries!
import numpy as np      # numpy is Python's "array" library
import pandas as pd     # Pandas is Python's "data" library ("dataframe" == spreadsheet)

In [19]:
# let's read in our births data...
# 
# for read_csv, use header=0 when row 0 is a header row
# 
cleaned_filename = "titanic_cleaned.csv"
df_tidy = pd.read_csv(cleaned_filename)   # encoding="utf-8" et al.
print(f"{cleaned_filename} : file read into a pandas dataframe.")
pd.options.display.min_rows = 30
df_tidy

titanic_cleaned.csv : file read into a pandas dataframe.


,pclass,survived,sex,age,sibsp,parch
0,1,1,0,29.0000,0,0
1,1,1,1,0.9167,1,2
2,1,0,0,2.0000,1,2
3,1,0,1,30.0000,1,2
4,1,0,0,25.0000,1,2
5,1,1,1,48.0000,0,0
6,1,1,0,63.0000,1,0
7,1,0,1,39.0000,0,0
8,1,1,0,53.0000,2,0
9,1,0,1,71.0000,0,0


In [20]:
df_tidy = df_tidy[['pclass', 'sex', 'age', 'sibsp', 'parch', 'survived']]

In [21]:
# First, look at the info:
df_tidy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1004 entries, 0 to 1266
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    1004 non-null   int64  
 1   sex       1004 non-null   int64  
 2   age       1004 non-null   float64
 3   sibsp     1004 non-null   int64  
 4   parch     1004 non-null   int64  
 5   survived  1004 non-null   int64  
dtypes: float64(1), int64(5)
memory usage: 54.9 KB


In [22]:
A = df_tidy.to_numpy()    # yields the underlying numpy array
A = A.astype('float64')  # so many types:  www.tutorialspoint.com/numpy/numpy_data_types.htm
print(A)

[[ 1.      0.     29.      0.      0.      1.    ]
 [ 1.      1.      0.9167  1.      2.      1.    ]
 [ 1.      0.      2.      1.      2.      0.    ]
 ...
 [ 3.      1.     26.5     0.      0.      0.    ]
 [ 3.      1.     27.      0.      0.      0.    ]
 [ 3.      1.     29.      0.      0.      0.    ]]


In [23]:
#
# Also, nice to have NUM_ROWS and NUM_COLS around
#
NUM_ROWS, NUM_COLS = A.shape
print(f"\nThe dataset has {NUM_ROWS} rows and {NUM_COLS} cols")


The dataset has 1004 rows and 6 cols


In [24]:
# choose a row index, n:
n = 42
COLUMNS = df_tidy.columns
CATEGORIES = ['pclass', 'survived', 'sex', 'age', 'sibsp', 'parch']
COL_INDEX = {}
for i, name in enumerate(COLUMNS):
    COL_INDEX[name] = i  # using the name (as key), look up the value (i)
print(f"COL_INDEX is {COL_INDEX}\n\n")
print(f"Passenger #{n} is {A[n]}")

for i in range(len(COLUMNS)):
    colname = COLUMNS[i]
    value = A[n][i]
    print(f"  Its {colname} is {value}")


COL_INDEX is {'pclass': 0, 'sex': 1, 'age': 2, 'sibsp': 3, 'parch': 4, 'survived': 5}


Passenger #42 is [ 1.  0. 53.  0.  0.  1.]
  Its pclass is 1.0
  Its sex is 0.0
  Its age is 53.0
  Its sibsp is 0.0
  Its parch is 0.0
  Its survived is 1.0


In [25]:
print("+++ Start of data definitions +++\n")

#
# we could do this at the data-frame level, too!
#

X_all = A[:, 0:-1] # X (features) all columns except 'survived'
y_all = A[:, -1]    # y (labels) ... is all rows, column 4 only

print(f"y_all (just the labels)   are \n {y_all}")
print(f"X_all (just the features) are \n {X_all}")

+++ Start of data definitions +++

y_all (just the labels)   are 
 [1. 1. 0. ... 0. 0. 0.]
X_all (just the features) are 
 [[ 1.      0.     29.      0.      0.    ]
 [ 1.      1.      0.9167  1.      2.    ]
 [ 1.      0.      2.      1.      2.    ]
 ...
 [ 3.      1.     26.5     0.      0.    ]
 [ 3.      1.     27.      0.      0.    ]
 [ 3.      1.     29.      0.      0.    ]]


In [26]:
#
# we scramble the data, to remove (potential) dependence on its ordering: 
# 
indices = np.random.permutation(len(y_all))  # indices is a permutation-list
# print(indices)

# we scramble both X and y, necessarily with the same permutation
X_labeled = X_all[indices]              # we apply the _same_ permutation to each!
y_labeled = y_all[indices]              # again...
print(f"The scrambled labels are \n {y_labeled}")
print(f"The corresponding data rows are \n {X_labeled}")

The scrambled labels are 
 [1. 0. 0. ... 1. 0. 0.]
The corresponding data rows are 
 [[ 3.   1.   6.   0.   1. ]
 [ 3.   1.  36.   1.   1. ]
 [ 3.   1.  38.5  0.   0. ]
 ...
 [ 2.   0.   7.   0.   2. ]
 [ 3.   0.  30.   1.   0. ]
 [ 1.   1.  37.   0.   1. ]]


In [27]:
#
# We next separate into test data and training data ... 
#    + We will train on the training data...
#    + We will _not_ look at the testing data to build the model
#
# Then, afterward, we will test on the testing data -- and see how well we do!
#

#
# a common convention:  train on 80%, test on 20%    Let's define the TEST_PERCENT
#

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2) # random_state=42

print(f"training with {len(y_train)} rows;  testing with {len(y_test)} rows\n" )

print(f"Held-out data... (testing data: {len(y_test)} rows)")
print(f"y_test: {y_test}")
print(f"X_test (first few rows): {X_test[0:5,:]}")  # 5 rows
print()
print(f"Data used for modeling... (training data: {len(y_train)} rows)")
print(f"y_train: {y_train}")
print(f"X_train (first few rows): {X_train[0:5,:]}")  # 5 rows

training with 803 rows;  testing with 201 rows

Held-out data... (testing data: 201 rows)
y_test: [1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0.
 1. 0. 1. 0. 1. 1. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0.
 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1.
 1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 0. 1. 1.
 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0.
 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0.
 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0.
 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 1. 0. 0. 0. 0.]
X_test (first few rows): [[ 1.  1. 28.  0.  0.]
 [ 1.  0. 33.  0.  0.]
 [ 3.  1. 25.  0.  0.]
 [ 2.  1. 25.  0.  0.]
 [ 1.  0. 27.  1.  2.]]

Data used for modeling... (training data: 803 rows)
y_train: [0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1.
 0. 0. 1. 1. 

In [28]:
#
# +++ This is the "Model-building and Model-training Cell"
#       
# Create a kNN model and train it! 
#
from sklearn.neighbors import KNeighborsClassifier

k = 84   # we don't know what k to use, so we guess!  (this will _not_ be a good value)
knn_model = KNeighborsClassifier(n_neighbors=k)       # here, k is the "k" in kNN

# we train the model (it's one line!)
knn_model.fit(X_train, y_train)                              # yay!  trained!
print("Created and trained a knn classifier with k =", k)  

Created and trained a knn classifier with k = 84


In [29]:
#
# +++ This cell is our "Model-testing Cell"
#
# Now, let's see how well our model does on our "held-out data" (the testing data)
#

# We run our test set:

# the function knn_model.predict is the instantiation of our model
# it's what runs the k-nearest-neighbors algorithm:
predicted_labels = knn_model.predict(X_test)      # THIS IS THE KEY LINE:  predict
actual_labels = y_test

# Let's print them so we can compare...
print("Predicted labels:", predicted_labels)
print("Actual  labels  :", actual_labels)

# And, some overall results
num_correct = sum(predicted_labels == actual_labels)
total = len(actual_labels)
print(f"\nResults on test set:  {num_correct} correct out of {total} total.")

Predicted labels: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Actual  labels  : [1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0.
 1. 0. 1. 0. 1. 1. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0.
 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1.
 1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 0. 1. 1.
 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0

In [30]:
#
# Let's print things in a vertical table
#

def compare_labels(predicted_labels, actual_labels):
    """ a more neatly formatted comparison """
    NUM_LABELS = len(predicted_labels)
    num_correct = 0

    print()
    print(f'row {"#":>3s} : {"predicted":>12s} {"actual":<12s}   {"result"}')   
    
    for i in range(NUM_LABELS):
        p = int(round(predicted_labels[i]))         # round protects from fp error 
        a = int(round(actual_labels[i]))
        result = "incorrect"
        if p == a:  # if they match,
            result = ""       # no longer incorrect
            num_correct += 1  # and we count a match!

        print(f"row {i:>3d} : {p:>12d} {a:<12d}   {result}")   

    print()
    print("Correct:", num_correct, "out of", NUM_LABELS)
    return num_correct

# let's try it out!
compare_labels(predicted_labels,actual_labels)


row   # :    predicted actual         result
row   0 :            0 1              incorrect
row   1 :            0 1              incorrect
row   2 :            0 0              
row   3 :            0 0              
row   4 :            0 1              incorrect
row   5 :            0 1              incorrect
row   6 :            0 1              incorrect
row   7 :            0 0              
row   8 :            0 0              
row   9 :            0 0              
row  10 :            0 0              
row  11 :            0 1              incorrect
row  12 :            0 0              
row  13 :            1 1              
row  14 :            0 0              
row  15 :            0 0              
row  16 :            1 1              
row  17 :            0 1              incorrect
row  18 :            0 0              
row  19 :            0 0              
row  20 :            0 0              
row  21 :            0 1              incorrect
row  22 :            0 0

130

In [31]:
def predictive_model( Features ):
    """ input: a list of two features 
                [ month, day ]
        output: the predicted outcome of 1 or 0 that correspond to "above" or "below" the median number of births
    """
    our_features = np.asarray([Features])                 # extra brackets needed
    predicted_births = knn_model.predict(our_features)   # PREDICT!
    return float(predicted_births)

#
# Try it!
# 
# Features = eval(input("Enter new Features: "))
#
Features = [9., 1., 0., 29., 0.]            
result = predictive_model( Features )
print(f"I predict {result} from Features {Features}")

I predict 1.0 from Features [9.0, 1.0, 0.0, 29.0, 0.0]


In [32]:
#
# Here, we use "cross validation" to find the "best" k...
#

from sklearn.model_selection import cross_val_score

#
# cross-validation splits the training set into two pieces:
#   + model-building and model-validation. We'll use "build" and "validate"
#
best_k = 84  # Not correct!
best_accuracy = 0.0  # also not correct...

# Note that we are cross-validating using only our TEST data!
for k in range(1,42):
    knn_cv_model = KNeighborsClassifier(n_neighbors=k)   # build a knn_model for every k
    cv_scores = cross_val_score( knn_cv_model, X_train, y_train, cv=5 )  # cv=5 means 80/20
    this_cv_accuracy = cv_scores.mean()               # mean() is numpy's built-in average function 
    print(f"k: {k:2d}  cv accuracy: {this_cv_accuracy:7.4f}")

    if this_cv_accuracy > best_accuracy:  
        best_accuracy = this_cv_accuracy  # track the best accuracy
        best_k = k                        # with the best k

    
# use best_k!
print(f"best_k = {best_k}   yields the highest average cv accuracy.")  # print the best one


k:  1  cv accuracy:  0.6925
k:  2  cv accuracy:  0.6912
k:  3  cv accuracy:  0.7149
k:  4  cv accuracy:  0.7198
k:  5  cv accuracy:  0.7161
k:  6  cv accuracy:  0.7273
k:  7  cv accuracy:  0.7323
k:  8  cv accuracy:  0.7024
k:  9  cv accuracy:  0.7236
k: 10  cv accuracy:  0.7086
k: 11  cv accuracy:  0.7261
k: 12  cv accuracy:  0.7049
k: 13  cv accuracy:  0.7087
k: 14  cv accuracy:  0.6974
k: 15  cv accuracy:  0.6974
k: 16  cv accuracy:  0.6912
k: 17  cv accuracy:  0.6987
k: 18  cv accuracy:  0.6974
k: 19  cv accuracy:  0.6937
k: 20  cv accuracy:  0.6837
k: 21  cv accuracy:  0.6825
k: 22  cv accuracy:  0.6862
k: 23  cv accuracy:  0.6875
k: 24  cv accuracy:  0.6825
k: 25  cv accuracy:  0.6838
k: 26  cv accuracy:  0.6800
k: 27  cv accuracy:  0.6788
k: 28  cv accuracy:  0.6750
k: 29  cv accuracy:  0.6763
k: 30  cv accuracy:  0.6676
k: 31  cv accuracy:  0.6651
k: 32  cv accuracy:  0.6563
k: 33  cv accuracy:  0.6526
k: 34  cv accuracy:  0.6526
k: 35  cv accuracy:  0.6576
k: 36  cv accuracy: 

In [33]:
#
# With the best k, we build and train a new model:
#
# Now using best_k instead of the original, randomly-guessed value:   
#
best_k = best_k   # not needed, but nice
from sklearn.neighbors import KNeighborsClassifier
knn_model_tuned = KNeighborsClassifier(n_neighbors=best_k)   # here, we use the best_k!

# we train the model (one line!)
knn_model_tuned.fit(X_train, y_train)                              # yay!  trained!
print(f"Created + trained a knn classifier, now tuned with a (best) k of {best_k}")  

# How does it do?!  The next cell will show...

Created + trained a knn classifier, now tuned with a (best) k of 7


In [34]:
#
# Re-create and re-run the  "Model-testing Cell"     How does it do with best_k?!
#
predicted_labels = knn_model_tuned.predict(X_test)
actual_labels = y_test

# Let's print them so we can compare...
print("Predicted labels:", predicted_labels)
print("Actual labels:", actual_labels)

# And, the overall results
num_correct = sum(predicted_labels == actual_labels)
total = len(actual_labels)
print(f"\nResults on test set:  {num_correct} correct out of {total} total.\n\n")

# Plus, we'll print our nicer table...
compare_labels(predicted_labels,actual_labels)

Predicted labels: [0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1.
 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 1. 1. 1.
 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0.
 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1.
 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0.
 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0.
 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 1. 0.
 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0.
 1. 0. 1. 0. 1. 0. 0. 0. 0.]
Actual labels: [1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0.
 1. 0. 1. 0. 1. 1. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0.
 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1.
 1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 0. 1. 1.
 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0

152

In [35]:
#
# Ok!  We tuned our knn modeling to use the "best" value of k...
#
# And, we should now use ALL available data to train our final predictive model:
#
knn_model_final = KNeighborsClassifier(n_neighbors=best_k)     # here, we use the best_k
knn_model_final.fit(X_all, y_all)                              # KEY DIFFERENCE:  we use ALL the data!
print(f"Created + trained a 'final' knn classifier, with a (best) k of {best_k}") 

Created + trained a 'final' knn classifier, with a (best) k of 7


In [36]:
#
# final predictive model (k-nearest-neighbor), with tuned k + ALL data incorporated
#

def predictive_model( Features, Model ):
    """ input: a list of birthday months and days
        output: the prediction of whether the birthday is above or below the mean (in integers)
    """
    our_features = np.asarray([Features])                 # extra brackets needed
    predicted_species = Model.predict(our_features)       # The model's prediction!
    predicted_species = int(round(predicted_species[0]))  # unpack the extra brackets
    return predicted_species
   
#
# Try it!
# 
# Features = eval(input("Enter new Features: "))
#

LoD = [[1., 1., 0., 29., 0.],
        [1., 1., 1., 0.9167, 1.],
        [3., 1., 23., 12., 2.],
        [2., 0., 17., 54., 2.],
        [3., 0., 42., 42., 2.]
]


# run on each one:
for Features in LoD:
    predicted_survival = predictive_model( Features, knn_model_final )  # pass in the model, too
    print(f"I predict {predicted_survival} from the features {Features}")    

I predict 0 from the features [1.0, 1.0, 0.0, 29.0, 0.0]
I predict 1 from the features [1.0, 1.0, 1.0, 0.9167, 1.0]
I predict 0 from the features [3.0, 1.0, 23.0, 12.0, 2.0]
I predict 0 from the features [2.0, 0.0, 17.0, 54.0, 2.0]
I predict 0 from the features [3.0, 0.0, 42.0, 42.0, 2.0]


##### Within 42 different cv tests, the best k came out to 7 with a cv accuracy of 0.7323. This is without any feature weighting at all... Now, we will move on to 3 different feature weightings below.

In [37]:
# Testing out the importance of 'sex' first...

df_tidy['sex'] *= 100
# df_tidy['sex'] /= 100
df_tidy

,pclass,sex,age,sibsp,parch,survived
0,1,0,29.0000,0,0,1
1,1,100,0.9167,1,2,1
2,1,0,2.0000,1,2,0
3,1,100,30.0000,1,2,0
4,1,0,25.0000,1,2,0
5,1,100,48.0000,0,0,1
6,1,0,63.0000,1,0,1
7,1,100,39.0000,0,0,0
8,1,0,53.0000,2,0,1
9,1,100,71.0000,0,0,0


In [38]:
#
A = df_tidy.to_numpy()    # yields the underlying numpy array
print(A)


[[  1.       0.      29.       0.       0.       1.    ]
 [  1.     100.       0.9167   1.       2.       1.    ]
 [  1.       0.       2.       1.       2.       0.    ]
 ...
 [  3.     100.      26.5      0.       0.       0.    ]
 [  3.     100.      27.       0.       0.       0.    ]
 [  3.     100.      29.       0.       0.       0.    ]]


In [39]:

#
# let's make sure it's all floating-point, so we can multiply and divide
#
A = A.astype('float64')  # so many types:  www.tutorialspoint.com/numpy/numpy_data_types.htm
print(A)


[[  1.       0.      29.       0.       0.       1.    ]
 [  1.     100.       0.9167   1.       2.       1.    ]
 [  1.       0.       2.       1.       2.       0.    ]
 ...
 [  3.     100.      26.5      0.       0.       0.    ]
 [  3.     100.      27.       0.       0.       0.    ]
 [  3.     100.      29.       0.       0.       0.    ]]


In [40]:
print("+++ Start of data definitions +++\n")

#
# we could do this at the data-frame level, too!
#

X_all = A[:, 0:-1]  # X (features) all columns except 'survived'
y_all = A[:, -1]    # y (labels) ... is all rows, column 4 only

print(f"y_all (just the labels)   are \n {y_all}")
print(f"X_all (just the features) are \n {X_all}")

+++ Start of data definitions +++

y_all (just the labels)   are 
 [1. 1. 0. ... 0. 0. 0.]
X_all (just the features) are 
 [[  1.       0.      29.       0.       0.    ]
 [  1.     100.       0.9167   1.       2.    ]
 [  1.       0.       2.       1.       2.    ]
 ...
 [  3.     100.      26.5      0.       0.    ]
 [  3.     100.      27.       0.       0.    ]
 [  3.     100.      29.       0.       0.    ]]


In [41]:
#
# we scramble the data, to remove (potential) dependence on its ordering: 
# 
indices = np.random.permutation(len(y_all))  # indices is a permutation-list

# we scramble both X and y, necessarily with the same permutation
X_labeled = X_all[indices]              # we apply the _same_ permutation to each!
y_labeled = y_all[indices]              # again...
print(f"The scrambled labels/species are \n {y_labeled}")
print(f"The corresponding data rows are \n {X_labeled}")


The scrambled labels/species are 
 [0. 0. 1. ... 0. 0. 1.]
The corresponding data rows are 
 [[  3. 100.  16.   0.   0.]
 [  3. 100.  41.   0.   0.]
 [  3. 100.  29.   3.   1.]
 ...
 [  1. 100.  55.   1.   1.]
 [  3. 100.  21.   0.   0.]
 [  1. 100.  36.   0.   0.]]


In [42]:
#
# We next separate into test data and training data ... 
#    + We will train on the training data...
#    + We will _not_ look at the testing data to build the model
#
# Then, afterward, we will test on the testing data -- and see how well we do!
#

#
# a common convention:  train on 80%, test on 20%    Let's define the TEST_PERCENT
#

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2) # random_state=42

print(f"training with {len(y_train)} rows;  testing with {len(y_test)} rows\n" )

print(f"Held-out data... (testing data: {len(y_test)} rows)")
print(f"y_test: {y_test}")
print(f"X_test (first few rows): {X_test[0:5,:]}")  # 5 rows
print()
print(f"Data used for modeling... (training data: {len(y_train)} rows)")
print(f"y_train: {y_train}")
print(f"X_train (first few rows): {X_train[0:5,:]}")  # 5 rows

training with 803 rows;  testing with 201 rows

Held-out data... (testing data: 201 rows)
y_test: [1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 0.
 1. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0.
 1. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 1.
 1. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0.
 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 0.
 1. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1.
 0. 0. 0. 1. 1. 0. 0. 0. 1.]
X_test (first few rows): [[  1.   0.  45.   0.   1.]
 [  3. 100.  36.   0.   0.]
 [  3. 100.  22.   0.   0.]
 [  3.   0.  37.   0.   0.]
 [  1.   0.  35.   0.   0.]]

Data used for modeling... (training data: 803 rows)
y_train: [1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 

In [43]:
#
# Here, we use "cross validation" to find the "best" k...
#

from sklearn.model_selection import cross_val_score

#
# cross-validation splits the training set into two pieces:
#   + model-building and model-validation. We'll use "build" and "validate"
#
best_k = 84  # Not correct!
best_accuracy = 0.0  # also not correct...

# Note that we are cross-validating using only our TEST data!
for k in range(1,42):
    knn_cv_model = KNeighborsClassifier(n_neighbors=k)   # build a knn_model for every k
    cv_scores = cross_val_score( knn_cv_model, X_train, y_train, cv=5 )  # cv=5 means 80/20
    this_cv_accuracy = cv_scores.mean()               # mean() is numpy's built-in average function 
    print(f"k: {k:2d}  cv accuracy: {this_cv_accuracy:7.4f}")

    if this_cv_accuracy > best_accuracy:  
        best_accuracy = this_cv_accuracy  # track the best accuracy
        best_k = k                        # with the best k

    
# use best_k!
print(f"best_k = {best_k}   yields the highest average cv accuracy.")  # print the best one


k:  1  cv accuracy:  0.7248
k:  2  cv accuracy:  0.7559
k:  3  cv accuracy:  0.7646
k:  4  cv accuracy:  0.7658
k:  5  cv accuracy:  0.7721
k:  6  cv accuracy:  0.7596
k:  7  cv accuracy:  0.7759
k:  8  cv accuracy:  0.7671
k:  9  cv accuracy:  0.7696
k: 10  cv accuracy:  0.7634
k: 11  cv accuracy:  0.7646
k: 12  cv accuracy:  0.7546
k: 13  cv accuracy:  0.7671
k: 14  cv accuracy:  0.7571
k: 15  cv accuracy:  0.7634
k: 16  cv accuracy:  0.7646
k: 17  cv accuracy:  0.7646
k: 18  cv accuracy:  0.7584
k: 19  cv accuracy:  0.7609
k: 20  cv accuracy:  0.7559
k: 21  cv accuracy:  0.7621
k: 22  cv accuracy:  0.7609
k: 23  cv accuracy:  0.7646
k: 24  cv accuracy:  0.7621
k: 25  cv accuracy:  0.7621
k: 26  cv accuracy:  0.7634
k: 27  cv accuracy:  0.7609
k: 28  cv accuracy:  0.7634
k: 29  cv accuracy:  0.7609
k: 30  cv accuracy:  0.7609
k: 31  cv accuracy:  0.7609
k: 32  cv accuracy:  0.7609
k: 33  cv accuracy:  0.7609
k: 34  cv accuracy:  0.7621
k: 35  cv accuracy:  0.7746
k: 36  cv accuracy: 

In [44]:
#
# With the best k, we build and train a new model:
#
# Now using best_k instead of the original, randomly-guessed value:   
#
best_k = best_k   # not needed, but nice
from sklearn.neighbors import KNeighborsClassifier
knn_model_tuned = KNeighborsClassifier(n_neighbors=best_k)   # here, we use the best_k!

# we train the model (one line!)
knn_model_tuned.fit(X_train, y_train)                              # yay!  trained!
print(f"Created + trained a knn classifier, now tuned with a (best) k of {best_k}")  

# How does it do?!  The next cell will show...

Created + trained a knn classifier, now tuned with a (best) k of 7


In [45]:
#
# Re-create and re-run the  "Model-testing Cell"     How does it do with best_k?!
#
predicted_labels = knn_model_tuned.predict(X_test)
actual_labels = y_test

# Let's print them so we can compare...
print("Predicted labels:", predicted_labels)
print("Actual labels:", actual_labels)

# And, the overall results
num_correct = sum(predicted_labels == actual_labels)
total = len(actual_labels)
print(f"\nResults on test set:  {num_correct} correct out of {total} total.\n\n")

# Plus, we'll print our nicer table...
compare_labels(predicted_labels,actual_labels)

Predicted labels: [1. 0. 0. 1. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1.
 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1.
 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0.
 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0. 0.
 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0.
 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1.
 0. 0. 0. 1. 1. 1. 0. 0. 1.]
Actual labels: [1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 0.
 1. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0.
 1. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1

160

##### When 'sex' is heavily considered, it doesn't change the overall cv and test accuracy much... In fact, the best k was 7 with a cv accuracy of 0.7759. In the test, this test got 7 more correct than the test without any feature weighting... (160/201).

In [46]:
df_tidy['sex'] /= 100       # Reset the weightings
df_tidy

,pclass,sex,age,sibsp,parch,survived
0,1,0.0,29.0000,0,0,1
1,1,1.0,0.9167,1,2,1
2,1,0.0,2.0000,1,2,0
3,1,1.0,30.0000,1,2,0
4,1,0.0,25.0000,1,2,0
5,1,1.0,48.0000,0,0,1
6,1,0.0,63.0000,1,0,1
7,1,1.0,39.0000,0,0,0
8,1,0.0,53.0000,2,0,1
9,1,1.0,71.0000,0,0,0


In [47]:
df_tidy['age'] *= 100      # Try another feature weighting
df_tidy

,pclass,sex,age,sibsp,parch,survived
0,1,0.0,2900.00,0,0,1
1,1,1.0,91.67,1,2,1
2,1,0.0,200.00,1,2,0
3,1,1.0,3000.00,1,2,0
4,1,0.0,2500.00,1,2,0
5,1,1.0,4800.00,0,0,1
6,1,0.0,6300.00,1,0,1
7,1,1.0,3900.00,0,0,0
8,1,0.0,5300.00,2,0,1
9,1,1.0,7100.00,0,0,0


In [48]:
#
A = df_tidy.to_numpy()    # yields the underlying numpy array
print(A)


[[1.000e+00 0.000e+00 2.900e+03 0.000e+00 0.000e+00 1.000e+00]
 [1.000e+00 1.000e+00 9.167e+01 1.000e+00 2.000e+00 1.000e+00]
 [1.000e+00 0.000e+00 2.000e+02 1.000e+00 2.000e+00 0.000e+00]
 ...
 [3.000e+00 1.000e+00 2.650e+03 0.000e+00 0.000e+00 0.000e+00]
 [3.000e+00 1.000e+00 2.700e+03 0.000e+00 0.000e+00 0.000e+00]
 [3.000e+00 1.000e+00 2.900e+03 0.000e+00 0.000e+00 0.000e+00]]


In [49]:

#
# let's make sure it's all floating-point, so we can multiply and divide
#
A = A.astype('float64')  # so many types:  www.tutorialspoint.com/numpy/numpy_data_types.htm
print(A)


[[1.000e+00 0.000e+00 2.900e+03 0.000e+00 0.000e+00 1.000e+00]
 [1.000e+00 1.000e+00 9.167e+01 1.000e+00 2.000e+00 1.000e+00]
 [1.000e+00 0.000e+00 2.000e+02 1.000e+00 2.000e+00 0.000e+00]
 ...
 [3.000e+00 1.000e+00 2.650e+03 0.000e+00 0.000e+00 0.000e+00]
 [3.000e+00 1.000e+00 2.700e+03 0.000e+00 0.000e+00 0.000e+00]
 [3.000e+00 1.000e+00 2.900e+03 0.000e+00 0.000e+00 0.000e+00]]


In [50]:
print("+++ Start of data definitions +++\n")

#
# we could do this at the data-frame level, too!
#

X_all = A[:, 0:-1]  # X (features) all columns except 'survived'
y_all = A[:, -1]    # y (labels) ... is all rows, column 4 only

print(f"y_all (just the labels)   are \n {y_all}")
print(f"X_all (just the features) are \n {X_all}")

+++ Start of data definitions +++

y_all (just the labels)   are 
 [1. 1. 0. ... 0. 0. 0.]
X_all (just the features) are 
 [[1.000e+00 0.000e+00 2.900e+03 0.000e+00 0.000e+00]
 [1.000e+00 1.000e+00 9.167e+01 1.000e+00 2.000e+00]
 [1.000e+00 0.000e+00 2.000e+02 1.000e+00 2.000e+00]
 ...
 [3.000e+00 1.000e+00 2.650e+03 0.000e+00 0.000e+00]
 [3.000e+00 1.000e+00 2.700e+03 0.000e+00 0.000e+00]
 [3.000e+00 1.000e+00 2.900e+03 0.000e+00 0.000e+00]]


In [51]:
#
# we scramble the data, to remove (potential) dependence on its ordering: 
# 
indices = np.random.permutation(len(y_all))  # indices is a permutation-list

# we scramble both X and y, necessarily with the same permutation
X_labeled = X_all[indices]              # we apply the _same_ permutation to each!
y_labeled = y_all[indices]              # again...
print(f"The scrambled labels/species are \n {y_labeled}")
print(f"The corresponding data rows are \n {X_labeled}")


The scrambled labels/species are 
 [1. 1. 0. ... 0. 0. 1.]
The corresponding data rows are 
 [[1.0e+00 1.0e+00 2.7e+03 0.0e+00 0.0e+00]
 [1.0e+00 1.0e+00 6.0e+02 0.0e+00 2.0e+00]
 [2.0e+00 1.0e+00 3.9e+03 0.0e+00 0.0e+00]
 ...
 [3.0e+00 1.0e+00 1.9e+03 0.0e+00 0.0e+00]
 [1.0e+00 1.0e+00 1.9e+03 1.0e+00 0.0e+00]
 [3.0e+00 1.0e+00 2.5e+03 0.0e+00 0.0e+00]]


In [52]:
#
# We next separate into test data and training data ... 
#    + We will train on the training data...
#    + We will _not_ look at the testing data to build the model
#
# Then, afterward, we will test on the testing data -- and see how well we do!
#

#
# a common convention:  train on 80%, test on 20%    Let's define the TEST_PERCENT
#

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2) # random_state=42

print(f"training with {len(y_train)} rows;  testing with {len(y_test)} rows\n" )

print(f"Held-out data... (testing data: {len(y_test)} rows)")
print(f"y_test: {y_test}")
print(f"X_test (first few rows): {X_test[0:5,:]}")  # 5 rows
print()
print(f"Data used for modeling... (training data: {len(y_train)} rows)")
print(f"y_train: {y_train}")
print(f"X_train (first few rows): {X_train[0:5,:]}")  # 5 rows

training with 803 rows;  testing with 201 rows

Held-out data... (testing data: 201 rows)
y_test: [1. 0. 0. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1.
 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1.
 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0.
 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1.
 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0.
 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 1. 1. 1. 0. 0.
 0. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0.
 0. 1. 1. 1. 1. 0. 0. 0. 1.]
X_test (first few rows): [[3.0e+00 0.0e+00 4.0e+02 0.0e+00 2.0e+00]
 [3.0e+00 1.0e+00 1.7e+03 0.0e+00 0.0e+00]
 [3.0e+00 1.0e+00 3.9e+03 0.0e+00 0.0e+00]
 [3.0e+00 0.0e+00 2.2e+03 0.0e+00 0.0e+00]
 [1.0e+00 1.0e+00 4.7e+03 0.0e+00 0.0e+00]]

Data used for modeling... (training data: 803 r

In [53]:
#
# Here, we use "cross validation" to find the "best" k...
#

from sklearn.model_selection import cross_val_score

#
# cross-validation splits the training set into two pieces:
#   + model-building and model-validation. We'll use "build" and "validate"
#
best_k = 84  # Not correct!
best_accuracy = 0.0  # also not correct...

# Note that we are cross-validating using only our TEST data!
for k in range(1,42):
    knn_cv_model = KNeighborsClassifier(n_neighbors=k)   # build a knn_model for every k
    cv_scores = cross_val_score( knn_cv_model, X_train, y_train, cv=5 )  # cv=5 means 80/20
    this_cv_accuracy = cv_scores.mean()               # mean() is numpy's built-in average function 
    print(f"k: {k:2d}  cv accuracy: {this_cv_accuracy:7.4f}")

    if this_cv_accuracy > best_accuracy:  
        best_accuracy = this_cv_accuracy  # track the best accuracy
        best_k = k                        # with the best k

    
# use best_k!
print(f"best_k = {best_k}   yields the highest average cv accuracy.")  # print the best one


k:  1  cv accuracy:  0.6837
k:  2  cv accuracy:  0.6575
k:  3  cv accuracy:  0.6625
k:  4  cv accuracy:  0.6500
k:  5  cv accuracy:  0.6475
k:  6  cv accuracy:  0.6525
k:  7  cv accuracy:  0.6401
k:  8  cv accuracy:  0.6351
k:  9  cv accuracy:  0.6314
k: 10  cv accuracy:  0.6414
k: 11  cv accuracy:  0.6202
k: 12  cv accuracy:  0.6351
k: 13  cv accuracy:  0.6264
k: 14  cv accuracy:  0.6301
k: 15  cv accuracy:  0.6214
k: 16  cv accuracy:  0.6314
k: 17  cv accuracy:  0.6214
k: 18  cv accuracy:  0.6214
k: 19  cv accuracy:  0.6127
k: 20  cv accuracy:  0.6326
k: 21  cv accuracy:  0.6264
k: 22  cv accuracy:  0.6289
k: 23  cv accuracy:  0.6277
k: 24  cv accuracy:  0.6214
k: 25  cv accuracy:  0.6239
k: 26  cv accuracy:  0.6202
k: 27  cv accuracy:  0.6215
k: 28  cv accuracy:  0.6264
k: 29  cv accuracy:  0.6264
k: 30  cv accuracy:  0.6339
k: 31  cv accuracy:  0.6451
k: 32  cv accuracy:  0.6401
k: 33  cv accuracy:  0.6376
k: 34  cv accuracy:  0.6252
k: 35  cv accuracy:  0.6252
k: 36  cv accuracy: 

In [54]:
#
# With the best k, we build and train a new model:
#
# Now using best_k instead of the original, randomly-guessed value:   
#
best_k = best_k   # not needed, but nice
from sklearn.neighbors import KNeighborsClassifier
knn_model_tuned = KNeighborsClassifier(n_neighbors=best_k)   # here, we use the best_k!

# we train the model (one line!)
knn_model_tuned.fit(X_train, y_train)                              # yay!  trained!
print(f"Created + trained a knn classifier, now tuned with a (best) k of {best_k}")  

# How does it do?!  The next cell will show...

Created + trained a knn classifier, now tuned with a (best) k of 1


In [55]:
#
# Re-create and re-run the  "Model-testing Cell"     How does it do with best_k?!
#
predicted_labels = knn_model_tuned.predict(X_test)
actual_labels = y_test

# Let's print them so we can compare...
print("Predicted labels:", predicted_labels)
print("Actual labels:", actual_labels)

# And, the overall results
num_correct = sum(predicted_labels == actual_labels)
total = len(actual_labels)
print(f"\nResults on test set:  {num_correct} correct out of {total} total.\n\n")

# Plus, we'll print our nicer table...
compare_labels(predicted_labels,actual_labels)

Predicted labels: [1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0.
 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0.
 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0.
 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0.
 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 1. 0. 1. 1. 1. 0. 0. 1. 0.]
Actual labels: [1. 0. 0. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1.
 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1.
 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0.
 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0

125

##### When 'age' is heavily considered, it actually drops the overall cv and testing accuracies. The best k is 1 with a cv accuracy of 0.6837. This is a medium drop from the test with no feature weightings. The testing accuracy came out to be 125/201...

##### We can see that considering gender more heavily leads to a higher cv and test accuracy rather than age...

In [56]:
df_tidy['age'] /= 100      # Reset!
df_tidy

,pclass,sex,age,sibsp,parch,survived
0,1,0.0,29.0000,0,0,1
1,1,1.0,0.9167,1,2,1
2,1,0.0,2.0000,1,2,0
3,1,1.0,30.0000,1,2,0
4,1,0.0,25.0000,1,2,0
5,1,1.0,48.0000,0,0,1
6,1,0.0,63.0000,1,0,1
7,1,1.0,39.0000,0,0,0
8,1,0.0,53.0000,2,0,1
9,1,1.0,71.0000,0,0,0


In [57]:
df_tidy['pclass'] *= 100      # Try another feature weighting
df_tidy

,pclass,sex,age,sibsp,parch,survived
0,100,0.0,29.0000,0,0,1
1,100,1.0,0.9167,1,2,1
2,100,0.0,2.0000,1,2,0
3,100,1.0,30.0000,1,2,0
4,100,0.0,25.0000,1,2,0
5,100,1.0,48.0000,0,0,1
6,100,0.0,63.0000,1,0,1
7,100,1.0,39.0000,0,0,0
8,100,0.0,53.0000,2,0,1
9,100,1.0,71.0000,0,0,0


In [58]:
#
A = df_tidy.to_numpy()    # yields the underlying numpy array
print(A)


[[100.       0.      29.       0.       0.       1.    ]
 [100.       1.       0.9167   1.       2.       1.    ]
 [100.       0.       2.       1.       2.       0.    ]
 ...
 [300.       1.      26.5      0.       0.       0.    ]
 [300.       1.      27.       0.       0.       0.    ]
 [300.       1.      29.       0.       0.       0.    ]]


In [59]:

#
# let's make sure it's all floating-point, so we can multiply and divide
#
A = A.astype('float64')  # so many types:  www.tutorialspoint.com/numpy/numpy_data_types.htm
print(A)


[[100.       0.      29.       0.       0.       1.    ]
 [100.       1.       0.9167   1.       2.       1.    ]
 [100.       0.       2.       1.       2.       0.    ]
 ...
 [300.       1.      26.5      0.       0.       0.    ]
 [300.       1.      27.       0.       0.       0.    ]
 [300.       1.      29.       0.       0.       0.    ]]


In [60]:
print("+++ Start of data definitions +++\n")

#
# we could do this at the data-frame level, too!
#

X_all = A[:, 0:-1] # X (features) all columns except 'survived'
y_all = A[:, -1]    # y (labels) ... is all rows, column 4 only

print(f"y_all (just the labels)   are \n {y_all}")
print(f"X_all (just the features) are \n {X_all}")

+++ Start of data definitions +++

y_all (just the labels)   are 
 [1. 1. 0. ... 0. 0. 0.]
X_all (just the features) are 
 [[100.       0.      29.       0.       0.    ]
 [100.       1.       0.9167   1.       2.    ]
 [100.       0.       2.       1.       2.    ]
 ...
 [300.       1.      26.5      0.       0.    ]
 [300.       1.      27.       0.       0.    ]
 [300.       1.      29.       0.       0.    ]]


In [61]:
#
# we scramble the data, to remove (potential) dependence on its ordering: 
# 
indices = np.random.permutation(len(y_all))  # indices is a permutation-list

# we scramble both X and y, necessarily with the same permutation
X_labeled = X_all[indices]              # we apply the _same_ permutation to each!
y_labeled = y_all[indices]              # again...
print(f"The scrambled labels/species are \n {y_labeled}")
print(f"The corresponding data rows are \n {X_labeled}")


The scrambled labels/species are 
 [0. 1. 0. ... 0. 0. 1.]
The corresponding data rows are 
 [[200.    1.   50.    0.    0. ]
 [300.    0.   19.    0.    0. ]
 [300.    1.   30.5   0.    0. ]
 ...
 [300.    0.   11.    4.    2. ]
 [100.    0.    2.    1.    2. ]
 [100.    0.   15.    0.    1. ]]


In [62]:
#
# We next separate into test data and training data ... 
#    + We will train on the training data...
#    + We will _not_ look at the testing data to build the model
#
# Then, afterward, we will test on the testing data -- and see how well we do!
#

#
# a common convention:  train on 80%, test on 20%    Let's define the TEST_PERCENT
#

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2) # random_state=42

print(f"training with {len(y_train)} rows;  testing with {len(y_test)} rows\n" )

print(f"Held-out data... (testing data: {len(y_test)} rows)")
print(f"y_test: {y_test}")
print(f"X_test (first few rows): {X_test[0:5,:]}")  # 5 rows
print()
print(f"Data used for modeling... (training data: {len(y_train)} rows)")
print(f"y_train: {y_train}")
print(f"X_train (first few rows): {X_train[0:5,:]}")  # 5 rows

training with 803 rows;  testing with 201 rows

Held-out data... (testing data: 201 rows)
y_test: [1. 1. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.
 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1.
 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0.
 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 1. 0.
 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0.
 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 1. 1.
 0. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 1. 0. 1. 0. 0. 1. 0. 0.]
X_test (first few rows): [[300.   1.  25.   0.   0.]
 [300.   0.   5.   2.   1.]
 [100.   1.  18.   1.   0.]
 [100.   0.  58.   0.   1.]
 [200.   1.  47.   0.   0.]]

Data used for modeling... (training data: 803 rows)
y_train: [0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 1. 0. 0. 0. 1. 1. 1. 0. 

In [63]:
#
# Here, we use "cross validation" to find the "best" k...
#

from sklearn.model_selection import cross_val_score

#
# cross-validation splits the training set into two pieces:
#   + model-building and model-validation. We'll use "build" and "validate"
#
best_k = 84  # Not correct!
best_accuracy = 0.0  # also not correct...

# Note that we are cross-validating using only our TEST data!
for k in range(1,42):
    knn_cv_model = KNeighborsClassifier(n_neighbors=k)   # build a knn_model for every k
    cv_scores = cross_val_score( knn_cv_model, X_train, y_train, cv=5 )  # cv=5 means 80/20
    this_cv_accuracy = cv_scores.mean()               # mean() is numpy's built-in average function 
    print(f"k: {k:2d}  cv accuracy: {this_cv_accuracy:7.4f}")

    if this_cv_accuracy > best_accuracy:  
        best_accuracy = this_cv_accuracy  # track the best accuracy
        best_k = k                        # with the best k

    
# use best_k!
print(f"best_k = {best_k}   yields the highest average cv accuracy.")  # print the best one


k:  1  cv accuracy:  0.6923
k:  2  cv accuracy:  0.7048
k:  3  cv accuracy:  0.7222
k:  4  cv accuracy:  0.7148
k:  5  cv accuracy:  0.7024
k:  6  cv accuracy:  0.7024
k:  7  cv accuracy:  0.7298
k:  8  cv accuracy:  0.7198
k:  9  cv accuracy:  0.7285
k: 10  cv accuracy:  0.7060
k: 11  cv accuracy:  0.7235
k: 12  cv accuracy:  0.7110
k: 13  cv accuracy:  0.7135
k: 14  cv accuracy:  0.7061
k: 15  cv accuracy:  0.7148
k: 16  cv accuracy:  0.6924
k: 17  cv accuracy:  0.7023
k: 18  cv accuracy:  0.6849
k: 19  cv accuracy:  0.6874
k: 20  cv accuracy:  0.6762
k: 21  cv accuracy:  0.6849
k: 22  cv accuracy:  0.6762
k: 23  cv accuracy:  0.6811
k: 24  cv accuracy:  0.6787
k: 25  cv accuracy:  0.6737
k: 26  cv accuracy:  0.6787
k: 27  cv accuracy:  0.6662
k: 28  cv accuracy:  0.6637
k: 29  cv accuracy:  0.6737
k: 30  cv accuracy:  0.6687
k: 31  cv accuracy:  0.6736
k: 32  cv accuracy:  0.6712
k: 33  cv accuracy:  0.6749
k: 34  cv accuracy:  0.6737
k: 35  cv accuracy:  0.6787
k: 36  cv accuracy: 

In [64]:
#
# With the best k, we build and train a new model:
#
# Now using best_k instead of the original, randomly-guessed value:   
#
best_k = best_k   # not needed, but nice
from sklearn.neighbors import KNeighborsClassifier
knn_model_tuned = KNeighborsClassifier(n_neighbors=best_k)   # here, we use the best_k!

# we train the model (one line!)
knn_model_tuned.fit(X_train, y_train)                              # yay!  trained!
print(f"Created + trained a knn classifier, now tuned with a (best) k of {best_k}")  

# How does it do?!  The next cell will show...

Created + trained a knn classifier, now tuned with a (best) k of 7


In [65]:
#
# Re-create and re-run the  "Model-testing Cell"     How does it do with best_k?!
#
predicted_labels = knn_model_tuned.predict(X_test)
actual_labels = y_test

# Let's print them so we can compare...
print("Predicted labels:", predicted_labels)
print("Actual labels:", actual_labels)

# And, the overall results
num_correct = sum(predicted_labels == actual_labels)
total = len(actual_labels)
print(f"\nResults on test set:  {num_correct} correct out of {total} total.\n\n")

# Plus, we'll print our nicer table...
compare_labels(predicted_labels,actual_labels)

Predicted labels: [0. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1.
 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1.
 0. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0.
 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0.
 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0.
 0. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0.
 0. 1. 0. 1. 0. 0. 0. 0. 0.]
Actual labels: [1. 1. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.
 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1.
 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0.
 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 1. 1. 0. 0

154

##### After increasing the feature weight for 'pclass' we can see that the best k is 7 with a cv accuracy of 0.7298... The testing accuracy is 154/201. This shows that having no feature weight at all yields the highest cv accuracy and testing accuracy. 

### Now, we will use a weight of 0 to test whether 'pclass' or 'age' is more important when determining Titanic-survival...

In [66]:
df_tidy['pclass'] /= 100      # Reset!
df_tidy

,pclass,sex,age,sibsp,parch,survived
0,1.0,0.0,29.0000,0,0,1
1,1.0,1.0,0.9167,1,2,1
2,1.0,0.0,2.0000,1,2,0
3,1.0,1.0,30.0000,1,2,0
4,1.0,0.0,25.0000,1,2,0
5,1.0,1.0,48.0000,0,0,1
6,1.0,0.0,63.0000,1,0,1
7,1.0,1.0,39.0000,0,0,0
8,1.0,0.0,53.0000,2,0,1
9,1.0,1.0,71.0000,0,0,0


In [67]:
df_tidy['pclass'] *= 0      # Try another feature weighting
df_tidy

,pclass,sex,age,sibsp,parch,survived
0,0.0,0.0,29.0000,0,0,1
1,0.0,1.0,0.9167,1,2,1
2,0.0,0.0,2.0000,1,2,0
3,0.0,1.0,30.0000,1,2,0
4,0.0,0.0,25.0000,1,2,0
5,0.0,1.0,48.0000,0,0,1
6,0.0,0.0,63.0000,1,0,1
7,0.0,1.0,39.0000,0,0,0
8,0.0,0.0,53.0000,2,0,1
9,0.0,1.0,71.0000,0,0,0


In [68]:
#
A = df_tidy.to_numpy()    # yields the underlying numpy array
print(A)


[[ 0.      0.     29.      0.      0.      1.    ]
 [ 0.      1.      0.9167  1.      2.      1.    ]
 [ 0.      0.      2.      1.      2.      0.    ]
 ...
 [ 0.      1.     26.5     0.      0.      0.    ]
 [ 0.      1.     27.      0.      0.      0.    ]
 [ 0.      1.     29.      0.      0.      0.    ]]


In [69]:

#
# let's make sure it's all floating-point, so we can multiply and divide
#
A = A.astype('float64')  # so many types:  www.tutorialspoint.com/numpy/numpy_data_types.htm
print(A)


[[ 0.      0.     29.      0.      0.      1.    ]
 [ 0.      1.      0.9167  1.      2.      1.    ]
 [ 0.      0.      2.      1.      2.      0.    ]
 ...
 [ 0.      1.     26.5     0.      0.      0.    ]
 [ 0.      1.     27.      0.      0.      0.    ]
 [ 0.      1.     29.      0.      0.      0.    ]]


In [70]:
print("+++ Start of data definitions +++\n")

#
# we could do this at the data-frame level, too!
#

X_all = A[:, 0:-1]  # X (features) all columns except 'survived'
y_all = A[:, -1]    # y (labels) ... is all rows, column 4 only

print(f"y_all (just the labels)   are \n {y_all}")
print(f"X_all (just the features) are \n {X_all}")

+++ Start of data definitions +++

y_all (just the labels)   are 
 [1. 1. 0. ... 0. 0. 0.]
X_all (just the features) are 
 [[ 0.      0.     29.      0.      0.    ]
 [ 0.      1.      0.9167  1.      2.    ]
 [ 0.      0.      2.      1.      2.    ]
 ...
 [ 0.      1.     26.5     0.      0.    ]
 [ 0.      1.     27.      0.      0.    ]
 [ 0.      1.     29.      0.      0.    ]]


In [71]:
#
# we scramble the data, to remove (potential) dependence on its ordering: 
# 
indices = np.random.permutation(len(y_all))  # indices is a permutation-list

# we scramble both X and y, necessarily with the same permutation
X_labeled = X_all[indices]              # we apply the _same_ permutation to each!
y_labeled = y_all[indices]              # again...
print(f"The scrambled labels/species are \n {y_labeled}")
print(f"The corresponding data rows are \n {X_labeled}")


The scrambled labels/species are 
 [0. 1. 0. ... 0. 0. 1.]
The corresponding data rows are 
 [[ 0.   1.  11.   5.   2. ]
 [ 0.   0.  24.   0.   2. ]
 [ 0.   1.  20.5  0.   0. ]
 ...
 [ 0.   1.  47.   0.   0. ]
 [ 0.   1.  18.   0.   0. ]
 [ 0.   0.  36.   0.   2. ]]


In [72]:
#
# We next separate into test data and training data ... 
#    + We will train on the training data...
#    + We will _not_ look at the testing data to build the model
#
# Then, afterward, we will test on the testing data -- and see how well we do!
#

#
# a common convention:  train on 80%, test on 20%    Let's define the TEST_PERCENT
#

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2) # random_state=42

print(f"training with {len(y_train)} rows;  testing with {len(y_test)} rows\n" )

print(f"Held-out data... (testing data: {len(y_test)} rows)")
print(f"y_test: {y_test}")
print(f"X_test (first few rows): {X_test[0:5,:]}")  # 5 rows
print()
print(f"Data used for modeling... (training data: {len(y_train)} rows)")
print(f"y_train: {y_train}")
print(f"X_train (first few rows): {X_train[0:5,:]}")  # 5 rows

training with 803 rows;  testing with 201 rows

Held-out data... (testing data: 201 rows)
y_test: [0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0.
 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1.
 0. 1. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 0.
 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 1. 1. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1.
 1. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0.
 1. 1. 1. 0. 0. 0. 0. 1. 0.]
X_test (first few rows): [[ 0.  1. 29.  0.  0.]
 [ 0.  0. 35.  1.  0.]
 [ 0.  0. 25.  1.  1.]
 [ 0.  1. 23.  0.  0.]
 [ 0.  1. 34.  1.  1.]]

Data used for modeling... (training data: 803 rows)
y_train: [0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0. 0. 1. 1.
 1. 0. 1. 0. 

In [73]:
#
# Here, we use "cross validation" to find the "best" k...
#

from sklearn.model_selection import cross_val_score

#
# cross-validation splits the training set into two pieces:
#   + model-building and model-validation. We'll use "build" and "validate"
#
best_k = 84  # Not correct!
best_accuracy = 0.0  # also not correct...

# Note that we are cross-validating using only our TEST data!
for k in range(1,42):
    knn_cv_model = KNeighborsClassifier(n_neighbors=k)   # build a knn_model for every k
    cv_scores = cross_val_score( knn_cv_model, X_train, y_train, cv=5 )  # cv=5 means 80/20
    this_cv_accuracy = cv_scores.mean()               # mean() is numpy's built-in average function 
    print(f"k: {k:2d}  cv accuracy: {this_cv_accuracy:7.4f}")

    if this_cv_accuracy > best_accuracy:  
        best_accuracy = this_cv_accuracy  # track the best accuracy
        best_k = k                        # with the best k

    
# use best_k!
print(f"best_k = {best_k}   yields the highest average cv accuracy.")  # print the best one


k:  1  cv accuracy:  0.6487
k:  2  cv accuracy:  0.6550
k:  3  cv accuracy:  0.6861
k:  4  cv accuracy:  0.6762
k:  5  cv accuracy:  0.7098
k:  6  cv accuracy:  0.6936
k:  7  cv accuracy:  0.6999
k:  8  cv accuracy:  0.6899
k:  9  cv accuracy:  0.7011
k: 10  cv accuracy:  0.6787
k: 11  cv accuracy:  0.7048
k: 12  cv accuracy:  0.6800
k: 13  cv accuracy:  0.6949
k: 14  cv accuracy:  0.6687
k: 15  cv accuracy:  0.6750
k: 16  cv accuracy:  0.6612
k: 17  cv accuracy:  0.6724
k: 18  cv accuracy:  0.6637
k: 19  cv accuracy:  0.6675
k: 20  cv accuracy:  0.6675
k: 21  cv accuracy:  0.6675
k: 22  cv accuracy:  0.6563
k: 23  cv accuracy:  0.6612
k: 24  cv accuracy:  0.6364
k: 25  cv accuracy:  0.6488
k: 26  cv accuracy:  0.6301
k: 27  cv accuracy:  0.6364
k: 28  cv accuracy:  0.6214
k: 29  cv accuracy:  0.6314
k: 30  cv accuracy:  0.6202
k: 31  cv accuracy:  0.6102
k: 32  cv accuracy:  0.6139
k: 33  cv accuracy:  0.6251
k: 34  cv accuracy:  0.6201
k: 35  cv accuracy:  0.6189
k: 36  cv accuracy: 

In [74]:
#
# With the best k, we build and train a new model:
#
# Now using best_k instead of the original, randomly-guessed value:   
#
best_k = best_k   # not needed, but nice
from sklearn.neighbors import KNeighborsClassifier
knn_model_tuned = KNeighborsClassifier(n_neighbors=best_k)   # here, we use the best_k!

# we train the model (one line!)
knn_model_tuned.fit(X_train, y_train)                              # yay!  trained!
print(f"Created + trained a knn classifier, now tuned with a (best) k of {best_k}")  

# How does it do?!  The next cell will show...

Created + trained a knn classifier, now tuned with a (best) k of 5


In [75]:
#
# Re-create and re-run the  "Model-testing Cell"     How does it do with best_k?!
#
predicted_labels = knn_model_tuned.predict(X_test)
actual_labels = y_test

# Let's print them so we can compare...
print("Predicted labels:", predicted_labels)
print("Actual labels:", actual_labels)

# And, the overall results
num_correct = sum(predicted_labels == actual_labels)
total = len(actual_labels)
print(f"\nResults on test set:  {num_correct} correct out of {total} total.\n\n")

# Plus, we'll print our nicer table...
compare_labels(predicted_labels,actual_labels)

Predicted labels: [0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1.
 1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1.
 1. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1.
 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 1.
 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 1.
 0. 0. 0. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 1.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1.
 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 1. 1. 0. 0. 0. 0. 0. 1. 1.]
Actual labels: [0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0.
 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1.
 0. 1. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 0.
 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0

143

### When pclass has a weight of 0, we see that the overall cv and testing accuracies drop when compared to the test with no feature weighting. The testing accuracy comes out to be 143 and the cv accuracy comes out to be 0.7098 (best k of 5). 

In [90]:
cleaned_filename = "titanic_cleaned.csv"
df_tidy = pd.read_csv(cleaned_filename)   # encoding="utf-8" et al.
print(f"{cleaned_filename} : file read into a pandas dataframe.")
pd.options.display.min_rows = 30
df_tidy

titanic_cleaned.csv : file read into a pandas dataframe.


,pclass,survived,sex,age,sibsp,parch
0,1,1,0,29.0000,0,0
1,1,1,1,0.9167,1,2
2,1,0,0,2.0000,1,2
3,1,0,1,30.0000,1,2
4,1,0,0,25.0000,1,2
5,1,1,1,48.0000,0,0
6,1,1,0,63.0000,1,0
7,1,0,1,39.0000,0,0
8,1,1,0,53.0000,2,0
9,1,0,1,71.0000,0,0


In [91]:
df_tidy = df_tidy[['pclass', 'sex', 'age', 'sibsp', 'parch', 'survived']]

In [92]:
df_tidy['age'] *= 0      # We test age now!
df_tidy

,pclass,sex,age,sibsp,parch,survived
0,1,0,0.0,0,0,1
1,1,1,0.0,1,2,1
2,1,0,0.0,1,2,0
3,1,1,0.0,1,2,0
4,1,0,0.0,1,2,0
5,1,1,0.0,0,0,1
6,1,0,0.0,1,0,1
7,1,1,0.0,0,0,0
8,1,0,0.0,2,0,1
9,1,1,0.0,0,0,0


In [79]:

#
# let's make sure it's all floating-point, so we can multiply and divide
#
A = A.astype('float64')  # so many types:  www.tutorialspoint.com/numpy/numpy_data_types.htm
print(A)


[[ 0.      0.     29.      0.      0.      1.    ]
 [ 0.      1.      0.9167  1.      2.      1.    ]
 [ 0.      0.      2.      1.      2.      0.    ]
 ...
 [ 0.      1.     26.5     0.      0.      0.    ]
 [ 0.      1.     27.      0.      0.      0.    ]
 [ 0.      1.     29.      0.      0.      0.    ]]


In [80]:

#
# let's make sure it's all floating-point, so we can multiply and divide
#
A = A.astype('float64')  # so many types:  www.tutorialspoint.com/numpy/numpy_data_types.htm
print(A)


[[ 0.      0.     29.      0.      0.      1.    ]
 [ 0.      1.      0.9167  1.      2.      1.    ]
 [ 0.      0.      2.      1.      2.      0.    ]
 ...
 [ 0.      1.     26.5     0.      0.      0.    ]
 [ 0.      1.     27.      0.      0.      0.    ]
 [ 0.      1.     29.      0.      0.      0.    ]]


In [81]:
print("+++ Start of data definitions +++\n")

#
# we could do this at the data-frame level, too!
#

X_all = A[:, 0:-1] # X (features) all columns except 'survived'
y_all = A[:, -1]    # y (labels) ... is all rows, column 4 only

print(f"y_all (just the labels)   are \n {y_all}")
print(f"X_all (just the features) are \n {X_all}")

+++ Start of data definitions +++

y_all (just the labels)   are 
 [1. 1. 0. ... 0. 0. 0.]
X_all (just the features) are 
 [[ 0.      0.     29.      0.      0.    ]
 [ 0.      1.      0.9167  1.      2.    ]
 [ 0.      0.      2.      1.      2.    ]
 ...
 [ 0.      1.     26.5     0.      0.    ]
 [ 0.      1.     27.      0.      0.    ]
 [ 0.      1.     29.      0.      0.    ]]


In [82]:
#
# we scramble the data, to remove (potential) dependence on its ordering: 
# 
indices = np.random.permutation(len(y_all))  # indices is a permutation-list

# we scramble both X and y, necessarily with the same permutation
X_labeled = X_all[indices]              # we apply the _same_ permutation to each!
y_labeled = y_all[indices]              # again...
print(f"The scrambled labels/species are \n {y_labeled}")
print(f"The corresponding data rows are \n {X_labeled}")


The scrambled labels/species are 
 [0. 0. 0. ... 0. 1. 1.]
The corresponding data rows are 
 [[ 0.  1. 42.  0.  0.]
 [ 0.  1. 29.  1.  0.]
 [ 0.  1. 42.  0.  0.]
 ...
 [ 0.  1. 41.  0.  0.]
 [ 0.  0. 18.  0.  0.]
 [ 0.  1. 31.  0.  0.]]


In [83]:
#
# We next separate into test data and training data ... 
#    + We will train on the training data...
#    + We will _not_ look at the testing data to build the model
#
# Then, afterward, we will test on the testing data -- and see how well we do!
#

#
# a common convention:  train on 80%, test on 20%    Let's define the TEST_PERCENT
#

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2) # random_state=42

print(f"training with {len(y_train)} rows;  testing with {len(y_test)} rows\n" )

print(f"Held-out data... (testing data: {len(y_test)} rows)")
print(f"y_test: {y_test}")
print(f"X_test (first few rows): {X_test[0:5,:]}")  # 5 rows
print()
print(f"Data used for modeling... (training data: {len(y_train)} rows)")
print(f"y_train: {y_train}")
print(f"X_train (first few rows): {X_train[0:5,:]}")  # 5 rows

training with 803 rows;  testing with 201 rows

Held-out data... (testing data: 201 rows)
y_test: [0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1.
 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 0.
 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1.
 1. 0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1.
 1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0.
 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 1. 0. 0. 1. 0. 1. 1. 0.]
X_test (first few rows): [[ 0.  1. 23.  0.  0.]
 [ 0.  1. 32.  0.  0.]
 [ 0.  1. 20.  0.  0.]
 [ 0.  0. 48.  1.  1.]
 [ 0.  1.  2.  1.  1.]]

Data used for modeling... (training data: 803 rows)
y_train: [1. 0. 0. 1. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1.
 1. 0. 0. 1. 

In [84]:
#
# Here, we use "cross validation" to find the "best" k...
#

from sklearn.model_selection import cross_val_score

#
# cross-validation splits the training set into two pieces:
#   + model-building and model-validation. We'll use "build" and "validate"
#
best_k = 84  # Not correct!
best_accuracy = 0.0  # also not correct...

# Note that we are cross-validating using only our TEST data!
for k in range(1,42):
    knn_cv_model = KNeighborsClassifier(n_neighbors=k)   # build a knn_model for every k
    cv_scores = cross_val_score( knn_cv_model, X_train, y_train, cv=5 )  # cv=5 means 80/20
    this_cv_accuracy = cv_scores.mean()               # mean() is numpy's built-in average function 
    print(f"k: {k:2d}  cv accuracy: {this_cv_accuracy:7.4f}")

    if this_cv_accuracy > best_accuracy:  
        best_accuracy = this_cv_accuracy  # track the best accuracy
        best_k = k                        # with the best k

    
# use best_k!
print(f"best_k = {best_k}   yields the highest average cv accuracy.")  # print the best one


k:  1  cv accuracy:  0.6562
k:  2  cv accuracy:  0.6638
k:  3  cv accuracy:  0.6812
k:  4  cv accuracy:  0.6662
k:  5  cv accuracy:  0.6899
k:  6  cv accuracy:  0.6899
k:  7  cv accuracy:  0.6986
k:  8  cv accuracy:  0.6899
k:  9  cv accuracy:  0.6999
k: 10  cv accuracy:  0.6887
k: 11  cv accuracy:  0.6874
k: 12  cv accuracy:  0.6799
k: 13  cv accuracy:  0.6762
k: 14  cv accuracy:  0.6737
k: 15  cv accuracy:  0.6738
k: 16  cv accuracy:  0.6675
k: 17  cv accuracy:  0.6688
k: 18  cv accuracy:  0.6576
k: 19  cv accuracy:  0.6576
k: 20  cv accuracy:  0.6563
k: 21  cv accuracy:  0.6526
k: 22  cv accuracy:  0.6501
k: 23  cv accuracy:  0.6538
k: 24  cv accuracy:  0.6439
k: 25  cv accuracy:  0.6401
k: 26  cv accuracy:  0.6489
k: 27  cv accuracy:  0.6439
k: 28  cv accuracy:  0.6414
k: 29  cv accuracy:  0.6414
k: 30  cv accuracy:  0.6376
k: 31  cv accuracy:  0.6326
k: 32  cv accuracy:  0.6302
k: 33  cv accuracy:  0.6289
k: 34  cv accuracy:  0.6277
k: 35  cv accuracy:  0.6252
k: 36  cv accuracy: 

In [85]:
#
# With the best k, we build and train a new model:
#
# Now using best_k instead of the original, randomly-guessed value:   
#
best_k = best_k   # not needed, but nice
from sklearn.neighbors import KNeighborsClassifier
knn_model_tuned = KNeighborsClassifier(n_neighbors=best_k)   # here, we use the best_k!

# we train the model (one line!)
knn_model_tuned.fit(X_train, y_train)                              # yay!  trained!
print(f"Created + trained a knn classifier, now tuned with a (best) k of {best_k}")  

# How does it do?!  The next cell will show...

Created + trained a knn classifier, now tuned with a (best) k of 9


In [86]:
#
# Re-create and re-run the  "Model-testing Cell"     How does it do with best_k?!
#
predicted_labels = knn_model_tuned.predict(X_test)
actual_labels = y_test

# Let's print them so we can compare...
print("Predicted labels:", predicted_labels)
print("Actual labels:", actual_labels)

# And, the overall results
num_correct = sum(predicted_labels == actual_labels)
total = len(actual_labels)
print(f"\nResults on test set:  {num_correct} correct out of {total} total.\n\n")

# Plus, we'll print our nicer table...
compare_labels(predicted_labels,actual_labels)

Predicted labels: [0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1.
 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0.
 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 1.
 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 1. 0. 0. 1. 1. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0.
 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 1. 0. 0. 1. 0.]
Actual labels: [0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1.
 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 0.
 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1.
 1. 0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0

152

##### When age is not considered at all, the testing accuracy comes out to be 152/201 which is higher than the testing accuracy when pclass is not considered. However, the cv accuracy for this test is lower than the one prior - the best k here is 9 with a cv accuracy of 0.6999. 




# Now, let's compare parch and sibsp to see which one has a more detrimental effect to the test and cv accuracies!

In [93]:
# let's read in our births data...
# 
# for read_csv, use header=0 when row 0 is a header row
# 
cleaned_filename = "titanic_cleaned.csv"
df_tidy = pd.read_csv(cleaned_filename)   # encoding="utf-8" et al.
print(f"{cleaned_filename} : file read into a pandas dataframe.")
pd.options.display.min_rows = 30
df_tidy

titanic_cleaned.csv : file read into a pandas dataframe.


,pclass,survived,sex,age,sibsp,parch
0,1,1,0,29.0000,0,0
1,1,1,1,0.9167,1,2
2,1,0,0,2.0000,1,2
3,1,0,1,30.0000,1,2
4,1,0,0,25.0000,1,2
5,1,1,1,48.0000,0,0
6,1,1,0,63.0000,1,0
7,1,0,1,39.0000,0,0
8,1,1,0,53.0000,2,0
9,1,0,1,71.0000,0,0


In [88]:
df_tidy['parch'] *= 0      # We test age now!
df_tidy

,pclass,survived,sex,age,sibsp,parch
0,1,1,0,29.0000,0,0
1,1,1,1,0.9167,1,0
2,1,0,0,2.0000,1,0
3,1,0,1,30.0000,1,0
4,1,0,0,25.0000,1,0
5,1,1,1,48.0000,0,0
6,1,1,0,63.0000,1,0
7,1,0,1,39.0000,0,0
8,1,1,0,53.0000,2,0
9,1,0,1,71.0000,0,0


In [94]:
df_tidy = df_tidy[['pclass', 'sex', 'age', 'sibsp', 'parch', 'survived']]

In [95]:

#
# let's make sure it's all floating-point, so we can multiply and divide
#
A = A.astype('float64')  # so many types:  www.tutorialspoint.com/numpy/numpy_data_types.htm
print(A)


[[ 0.      0.     29.      0.      0.      1.    ]
 [ 0.      1.      0.9167  1.      2.      1.    ]
 [ 0.      0.      2.      1.      2.      0.    ]
 ...
 [ 0.      1.     26.5     0.      0.      0.    ]
 [ 0.      1.     27.      0.      0.      0.    ]
 [ 0.      1.     29.      0.      0.      0.    ]]


In [96]:
print("+++ Start of data definitions +++\n")

#
# we could do this at the data-frame level, too!
#

X_all = A[:, 0:-1] # X (features) all columns except 'survived'
y_all = A[:, -1]    # y (labels) ... is all rows, column 4 only

print(f"y_all (just the labels)   are \n {y_all}")
print(f"X_all (just the features) are \n {X_all}")

+++ Start of data definitions +++

y_all (just the labels)   are 
 [1. 1. 0. ... 0. 0. 0.]
X_all (just the features) are 
 [[ 0.      0.     29.      0.      0.    ]
 [ 0.      1.      0.9167  1.      2.    ]
 [ 0.      0.      2.      1.      2.    ]
 ...
 [ 0.      1.     26.5     0.      0.    ]
 [ 0.      1.     27.      0.      0.    ]
 [ 0.      1.     29.      0.      0.    ]]


In [97]:
#
# we scramble the data, to remove (potential) dependence on its ordering: 
# 
indices = np.random.permutation(len(y_all))  # indices is a permutation-list

# we scramble both X and y, necessarily with the same permutation
X_labeled = X_all[indices]              # we apply the _same_ permutation to each!
y_labeled = y_all[indices]              # again...
print(f"The scrambled labels/species are \n {y_labeled}")
print(f"The corresponding data rows are \n {X_labeled}")


The scrambled labels/species are 
 [0. 0. 0. ... 0. 0. 1.]
The corresponding data rows are 
 [[ 0.  1. 16.  0.  0.]
 [ 0.  1. 26.  0.  0.]
 [ 0.  0. 29.  0.  0.]
 ...
 [ 0.  0. 28.  0.  0.]
 [ 0.  1. 46.  0.  0.]
 [ 0.  0. 55.  0.  0.]]


In [98]:
#
# We next separate into test data and training data ... 
#    + We will train on the training data...
#    + We will _not_ look at the testing data to build the model
#
# Then, afterward, we will test on the testing data -- and see how well we do!
#

#
# a common convention:  train on 80%, test on 20%    Let's define the TEST_PERCENT
#

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2) # random_state=42

print(f"training with {len(y_train)} rows;  testing with {len(y_test)} rows\n" )

print(f"Held-out data... (testing data: {len(y_test)} rows)")
print(f"y_test: {y_test}")
print(f"X_test (first few rows): {X_test[0:5,:]}")  # 5 rows
print()
print(f"Data used for modeling... (training data: {len(y_train)} rows)")
print(f"y_train: {y_train}")
print(f"X_train (first few rows): {X_train[0:5,:]}")  # 5 rows

training with 803 rows;  testing with 201 rows

Held-out data... (testing data: 201 rows)
y_test: [0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0.
 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 1. 1. 1. 0. 1.
 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1.
 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0.
 0. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 1. 1.
 0. 1. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1.
 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 1.
 1. 1. 0. 1. 0. 0. 0. 0. 0.]
X_test (first few rows): [[ 0.  1. 54.  0.  0.]
 [ 0.  1. 47.  0.  0.]
 [ 0.  0. 38.  0.  0.]
 [ 0.  1. 19.  0.  0.]
 [ 0.  1. 19.  1.  1.]]

Data used for modeling... (training data: 803 rows)
y_train: [0. 1. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0.
 0. 1. 1. 1. 

In [99]:
#
# Here, we use "cross validation" to find the "best" k...
#

from sklearn.model_selection import cross_val_score

#
# cross-validation splits the training set into two pieces:
#   + model-building and model-validation. We'll use "build" and "validate"
#
best_k = 84  # Not correct!
best_accuracy = 0.0  # also not correct...

# Note that we are cross-validating using only our TEST data!
for k in range(1,42):
    knn_cv_model = KNeighborsClassifier(n_neighbors=k)   # build a knn_model for every k
    cv_scores = cross_val_score( knn_cv_model, X_train, y_train, cv=5 )  # cv=5 means 80/20
    this_cv_accuracy = cv_scores.mean()               # mean() is numpy's built-in average function 
    print(f"k: {k:2d}  cv accuracy: {this_cv_accuracy:7.4f}")

    if this_cv_accuracy > best_accuracy:  
        best_accuracy = this_cv_accuracy  # track the best accuracy
        best_k = k                        # with the best k

    
# use best_k!
print(f"best_k = {best_k}   yields the highest average cv accuracy.")  # print the best one


k:  1  cv accuracy:  0.6700
k:  2  cv accuracy:  0.6800
k:  3  cv accuracy:  0.6924
k:  4  cv accuracy:  0.6687
k:  5  cv accuracy:  0.7186
k:  6  cv accuracy:  0.6912
k:  7  cv accuracy:  0.7086
k:  8  cv accuracy:  0.7036
k:  9  cv accuracy:  0.6962
k: 10  cv accuracy:  0.6862
k: 11  cv accuracy:  0.6937
k: 12  cv accuracy:  0.6912
k: 13  cv accuracy:  0.6887
k: 14  cv accuracy:  0.6837
k: 15  cv accuracy:  0.6825
k: 16  cv accuracy:  0.6738
k: 17  cv accuracy:  0.6800
k: 18  cv accuracy:  0.6701
k: 19  cv accuracy:  0.6813
k: 20  cv accuracy:  0.6651
k: 21  cv accuracy:  0.6688
k: 22  cv accuracy:  0.6688
k: 23  cv accuracy:  0.6626
k: 24  cv accuracy:  0.6700
k: 25  cv accuracy:  0.6676
k: 26  cv accuracy:  0.6563
k: 27  cv accuracy:  0.6476
k: 28  cv accuracy:  0.6526
k: 29  cv accuracy:  0.6464
k: 30  cv accuracy:  0.6376
k: 31  cv accuracy:  0.6376
k: 32  cv accuracy:  0.6339
k: 33  cv accuracy:  0.6302
k: 34  cv accuracy:  0.6438
k: 35  cv accuracy:  0.6277
k: 36  cv accuracy: 

In [100]:
#
# With the best k, we build and train a new model:
#
# Now using best_k instead of the original, randomly-guessed value:   
#
best_k = best_k   # not needed, but nice
from sklearn.neighbors import KNeighborsClassifier
knn_model_tuned = KNeighborsClassifier(n_neighbors=best_k)   # here, we use the best_k!

# we train the model (one line!)
knn_model_tuned.fit(X_train, y_train)                              # yay!  trained!
print(f"Created + trained a knn classifier, now tuned with a (best) k of {best_k}")  

# How does it do?!  The next cell will show...

Created + trained a knn classifier, now tuned with a (best) k of 5


In [101]:
#
# Re-create and re-run the  "Model-testing Cell"     How does it do with best_k?!
#
predicted_labels = knn_model_tuned.predict(X_test)
actual_labels = y_test

# Let's print them so we can compare...
print("Predicted labels:", predicted_labels)
print("Actual labels:", actual_labels)

# And, the overall results
num_correct = sum(predicted_labels == actual_labels)
total = len(actual_labels)
print(f"\nResults on test set:  {num_correct} correct out of {total} total.\n\n")

# Plus, we'll print our nicer table...
compare_labels(predicted_labels,actual_labels)

Predicted labels: [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1.
 1. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1.
 0. 0. 1. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0.
 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0.
 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 0. 1. 0. 1. 0. 1.
 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0.
 0. 0. 1. 1. 1. 0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0.
 1. 1. 0. 1. 0. 0. 0. 0. 0.]
Actual labels: [0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0.
 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 1. 1. 1. 0. 1.
 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1.
 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1

149

##### When 'parch' is not considered, we see that the testing accuracy comes out to be 149/201, the best k is 5, and the cv accuracy is 0.7186... Now moving on to seeing what happens when the weight for sibsp is 0...

In [102]:
# let's read in our births data...
# 
# for read_csv, use header=0 when row 0 is a header row
# 
cleaned_filename = "titanic_cleaned.csv"
df_tidy = pd.read_csv(cleaned_filename)   # encoding="utf-8" et al.
print(f"{cleaned_filename} : file read into a pandas dataframe.")
pd.options.display.min_rows = 30
df_tidy

titanic_cleaned.csv : file read into a pandas dataframe.


,pclass,survived,sex,age,sibsp,parch
0,1,1,0,29.0000,0,0
1,1,1,1,0.9167,1,2
2,1,0,0,2.0000,1,2
3,1,0,1,30.0000,1,2
4,1,0,0,25.0000,1,2
5,1,1,1,48.0000,0,0
6,1,1,0,63.0000,1,0
7,1,0,1,39.0000,0,0
8,1,1,0,53.0000,2,0
9,1,0,1,71.0000,0,0


In [103]:
df_tidy['sibsp'] *= 0      # We test age now!
df_tidy

,pclass,survived,sex,age,sibsp,parch
0,1,1,0,29.0000,0,0
1,1,1,1,0.9167,0,2
2,1,0,0,2.0000,0,2
3,1,0,1,30.0000,0,2
4,1,0,0,25.0000,0,2
5,1,1,1,48.0000,0,0
6,1,1,0,63.0000,0,0
7,1,0,1,39.0000,0,0
8,1,1,0,53.0000,0,0
9,1,0,1,71.0000,0,0


In [104]:
df_tidy = df_tidy[['pclass', 'sex', 'age', 'sibsp', 'parch', 'survived']]

In [105]:

#
# let's make sure it's all floating-point, so we can multiply and divide
#
A = A.astype('float64')  # so many types:  www.tutorialspoint.com/numpy/numpy_data_types.htm
print(A)


[[ 0.      0.     29.      0.      0.      1.    ]
 [ 0.      1.      0.9167  1.      2.      1.    ]
 [ 0.      0.      2.      1.      2.      0.    ]
 ...
 [ 0.      1.     26.5     0.      0.      0.    ]
 [ 0.      1.     27.      0.      0.      0.    ]
 [ 0.      1.     29.      0.      0.      0.    ]]


In [106]:
print("+++ Start of data definitions +++\n")

#
# we could do this at the data-frame level, too!
#

X_all = A[:, 0:-1] # X (features) all columns except 'survived'
y_all = A[:, -1]    # y (labels) ... is all rows, column 4 only

print(f"y_all (just the labels)   are \n {y_all}")
print(f"X_all (just the features) are \n {X_all}")

+++ Start of data definitions +++

y_all (just the labels)   are 
 [1. 1. 0. ... 0. 0. 0.]
X_all (just the features) are 
 [[ 0.      0.     29.      0.      0.    ]
 [ 0.      1.      0.9167  1.      2.    ]
 [ 0.      0.      2.      1.      2.    ]
 ...
 [ 0.      1.     26.5     0.      0.    ]
 [ 0.      1.     27.      0.      0.    ]
 [ 0.      1.     29.      0.      0.    ]]


In [107]:
#
# we scramble the data, to remove (potential) dependence on its ordering: 
# 
indices = np.random.permutation(len(y_all))  # indices is a permutation-list

# we scramble both X and y, necessarily with the same permutation
X_labeled = X_all[indices]              # we apply the _same_ permutation to each!
y_labeled = y_all[indices]              # again...
print(f"The scrambled labels/species are \n {y_labeled}")
print(f"The corresponding data rows are \n {X_labeled}")


The scrambled labels/species are 
 [0. 1. 0. ... 0. 1. 1.]
The corresponding data rows are 
 [[ 0.  1. 27.  0.  0.]
 [ 0.  0.  5.  4.  2.]
 [ 0.  1. 26.  0.  0.]
 ...
 [ 0.  1. 25.  0.  0.]
 [ 0.  0. 26.  0.  0.]
 [ 0.  1. 11.  1.  2.]]


In [108]:
#
# We next separate into test data and training data ... 
#    + We will train on the training data...
#    + We will _not_ look at the testing data to build the model
#
# Then, afterward, we will test on the testing data -- and see how well we do!
#

#
# a common convention:  train on 80%, test on 20%    Let's define the TEST_PERCENT
#

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2) # random_state=42

print(f"training with {len(y_train)} rows;  testing with {len(y_test)} rows\n" )

print(f"Held-out data... (testing data: {len(y_test)} rows)")
print(f"y_test: {y_test}")
print(f"X_test (first few rows): {X_test[0:5,:]}")  # 5 rows
print()
print(f"Data used for modeling... (training data: {len(y_train)} rows)")
print(f"y_train: {y_train}")
print(f"X_train (first few rows): {X_train[0:5,:]}")  # 5 rows

training with 803 rows;  testing with 201 rows

Held-out data... (testing data: 201 rows)
y_test: [1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1.
 0. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0.
 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0. 0. 0.
 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0.
 1. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0.
 1. 0. 1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 1.
 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 1. 1. 0. 0. 0. 1. 1. 0.]
X_test (first few rows): [[ 0.  0. 18.  0.  0.]
 [ 0.  0. 31.  1.  1.]
 [ 0.  0. 32.  0.  0.]
 [ 0.  1. 35.  0.  0.]
 [ 0.  1. 32.  1.  0.]]

Data used for modeling... (training data: 803 rows)
y_train: [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0.
 0. 1. 1. 0. 

In [109]:
#
# Here, we use "cross validation" to find the "best" k...
#

from sklearn.model_selection import cross_val_score

#
# cross-validation splits the training set into two pieces:
#   + model-building and model-validation. We'll use "build" and "validate"
#
best_k = 84  # Not correct!
best_accuracy = 0.0  # also not correct...

# Note that we are cross-validating using only our TEST data!
for k in range(1,42):
    knn_cv_model = KNeighborsClassifier(n_neighbors=k)   # build a knn_model for every k
    cv_scores = cross_val_score( knn_cv_model, X_train, y_train, cv=5 )  # cv=5 means 80/20
    this_cv_accuracy = cv_scores.mean()               # mean() is numpy's built-in average function 
    print(f"k: {k:2d}  cv accuracy: {this_cv_accuracy:7.4f}")

    if this_cv_accuracy > best_accuracy:  
        best_accuracy = this_cv_accuracy  # track the best accuracy
        best_k = k                        # with the best k

    
# use best_k!
print(f"best_k = {best_k}   yields the highest average cv accuracy.")  # print the best one


k:  1  cv accuracy:  0.6925
k:  2  cv accuracy:  0.6812
k:  3  cv accuracy:  0.7023
k:  4  cv accuracy:  0.6911
k:  5  cv accuracy:  0.7111
k:  6  cv accuracy:  0.6861
k:  7  cv accuracy:  0.7036
k:  8  cv accuracy:  0.6912
k:  9  cv accuracy:  0.7024
k: 10  cv accuracy:  0.6887
k: 11  cv accuracy:  0.6824
k: 12  cv accuracy:  0.6799
k: 13  cv accuracy:  0.6862
k: 14  cv accuracy:  0.6812
k: 15  cv accuracy:  0.6961
k: 16  cv accuracy:  0.6849
k: 17  cv accuracy:  0.6875
k: 18  cv accuracy:  0.6762
k: 19  cv accuracy:  0.6837
k: 20  cv accuracy:  0.6675
k: 21  cv accuracy:  0.6675
k: 22  cv accuracy:  0.6638
k: 23  cv accuracy:  0.6713
k: 24  cv accuracy:  0.6650
k: 25  cv accuracy:  0.6663
k: 26  cv accuracy:  0.6538
k: 27  cv accuracy:  0.6563
k: 28  cv accuracy:  0.6488
k: 29  cv accuracy:  0.6576
k: 30  cv accuracy:  0.6414
k: 31  cv accuracy:  0.6488
k: 32  cv accuracy:  0.6364
k: 33  cv accuracy:  0.6464
k: 34  cv accuracy:  0.6302
k: 35  cv accuracy:  0.6364
k: 36  cv accuracy: 

In [110]:
#
# With the best k, we build and train a new model:
#
# Now using best_k instead of the original, randomly-guessed value:   
#
best_k = best_k   # not needed, but nice
from sklearn.neighbors import KNeighborsClassifier
knn_model_tuned = KNeighborsClassifier(n_neighbors=best_k)   # here, we use the best_k!

# we train the model (one line!)
knn_model_tuned.fit(X_train, y_train)                              # yay!  trained!
print(f"Created + trained a knn classifier, now tuned with a (best) k of {best_k}")  

# How does it do?!  The next cell will show...

Created + trained a knn classifier, now tuned with a (best) k of 5


In [111]:
#
# Re-create and re-run the  "Model-testing Cell"     How does it do with best_k?!
#
predicted_labels = knn_model_tuned.predict(X_test)
actual_labels = y_test

# Let's print them so we can compare...
print("Predicted labels:", predicted_labels)
print("Actual labels:", actual_labels)

# And, the overall results
num_correct = sum(predicted_labels == actual_labels)
total = len(actual_labels)
print(f"\nResults on test set:  {num_correct} correct out of {total} total.\n\n")

# Plus, we'll print our nicer table...
compare_labels(predicted_labels,actual_labels)

Predicted labels: [1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1.
 1. 0. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0.
 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1.
 0. 0. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0.
 1. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0.
 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 1. 1. 1. 0. 0. 1. 0. 0.]
Actual labels: [1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1.
 0. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0.
 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0. 0. 0.
 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0

146

##### As we can see, the best k turned out to be 5 with a cv accuracy of 0.7111. The testing accuracy comes out to be 146/201.

##### The results of the two tests above are so similar so our program cannot determine which feature (sibsp or parch) is more detrimental to the survival results.

##### From my (human perspective), I can see how having siblings and parents onboard can influence the survival of a person. They might try to save others before themselves or not. If the siblings or parents are not on board, then I believe these two features don't matter at all. With that being said, I don't believe these two features are as important to consider than age. 

In [112]:
# let's read in our Other Titanic Passengers data...
# 
# for read_csv, use header=0 when row 0 is a header row
# 
cleaned_filename = "Other_Titanic_passengers.csv"
df_tidy = pd.read_csv(cleaned_filename)   # encoding="utf-8" et al.
print(f"{cleaned_filename} : file read into a pandas dataframe.")
pd.options.display.min_rows = 30
df_tidy

Other_Titanic_passengers.csv : file read into a pandas dataframe.


,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,3,0,"Ali, Mr. Ahmed",male,24.0,0,0,SOTON/O.Q. 3101311,7.0500,NaN,S,NaN,NaN,NaN
1,3,0,"Ali, Mr. William",male,25.0,0,0,SOTON/O.Q. 3101312,7.0500,NaN,S,NaN,79.0,Argentina
2,3,0,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,NaN,NaN,"Lower Clapton, Middlesex or Erdington, Birmingham"
3,3,0,"Allum, Mr. Owen George",male,18.0,0,0,2223,8.3000,NaN,S,NaN,259.0,"Windsor, England New York, NY"
4,3,1,"Andersen-Jensen, Miss. Carla Christine Nielsine",female,19.0,1,0,350046,7.8542,NaN,S,16.0,NaN,NaN
5,3,0,"Andersen, Mr. Albert Karvin",male,32.0,0,0,C 4001,22.5250,NaN,S,NaN,260.0,"Bergen, Norway"
6,1,0,"Baxter, Mr. Quigg Edmond",male,24.0,0,1,PC 17558,247.5208,B58 B60,C,NaN,NaN,"Montreal, PQ"
7,1,1,"Baxter, Mrs. James (Helene DeLaudeniere Chaput)",female,50.0,0,1,PC 17558,247.5208,B58 B60,C,6.0,NaN,"Montreal, PQ"
8,2,1,"Becker, Master. Richard F",male,1.0,2,1,230136,39.0000,F4,S,11.0,NaN,"Guntur, India / Benton Harbour, MI"
9,2,1,"Becker, Miss. Marion Louise",female,4.0,2,1,230136,39.0000,F4,S,11.0,NaN,"Guntur, India / Benton Harbour, MI"


In [115]:
# Let's look at the dataframe's columns:
df = df_tidy
df.columns

Index(['pclass', 'survived', 'name', 'sex', 'age', 'sibsp', 'parch', 'ticket',
       'fare', 'cabin', 'embarked', 'boat', 'body', 'home.dest'],
      dtype='object')

In [116]:
# we can drop a series of data (a row or a column)
# the dimensions each have a numeric value, row~0, col~1, but let's use readable names we define:
ROW = 0
COLUMN = 1

df_clean1 = df.drop('home.dest', axis=COLUMN)
df_clean1

# df_clean1 is a new dataframe, without that unwanted column

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body
0,3,0,"Ali, Mr. Ahmed",male,24.0,0,0,SOTON/O.Q. 3101311,7.0500,NaN,S,NaN,NaN
1,3,0,"Ali, Mr. William",male,25.0,0,0,SOTON/O.Q. 3101312,7.0500,NaN,S,NaN,79.0
2,3,0,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,NaN,NaN
3,3,0,"Allum, Mr. Owen George",male,18.0,0,0,2223,8.3000,NaN,S,NaN,259.0
4,3,1,"Andersen-Jensen, Miss. Carla Christine Nielsine",female,19.0,1,0,350046,7.8542,NaN,S,16.0,NaN
5,3,0,"Andersen, Mr. Albert Karvin",male,32.0,0,0,C 4001,22.5250,NaN,S,NaN,260.0
6,1,0,"Baxter, Mr. Quigg Edmond",male,24.0,0,1,PC 17558,247.5208,B58 B60,C,NaN,NaN
7,1,1,"Baxter, Mrs. James (Helene DeLaudeniere Chaput)",female,50.0,0,1,PC 17558,247.5208,B58 B60,C,6.0,NaN
8,2,1,"Becker, Master. Richard F",male,1.0,2,1,230136,39.0000,F4,S,11.0,NaN
9,2,1,"Becker, Miss. Marion Louise",female,4.0,2,1,230136,39.0000,F4,S,11.0,NaN


In [119]:
# we can drop a series of data (a row or a column)
# the dimensions each have a numeric value, row~0, col~1, but let's use readable names we define:
ROW = 0
COLUMN = 1

df_clean2 = df_clean1.drop('body', axis=COLUMN)
df_clean3 = df_clean2.drop('boat', axis=COLUMN)
df_clean4 = df_clean3.drop('embarked', axis=COLUMN)
df_clean5 = df_clean4.drop('cabin', axis=COLUMN)
df_clean6 = df_clean5.drop('fare', axis=COLUMN)
df_clean7 = df_clean6.drop('ticket', axis=COLUMN)
df_clean8 = df_clean7.drop('name', axis=COLUMN)
df_tidy = df_clean8
df_tidy

# df_clean1 is a new dataframe, without that unwanted column

,pclass,survived,sex,age,sibsp,parch
0,3,0,male,24.0,0,0
1,3,0,male,25.0,0,0
2,3,0,male,35.0,0,0
3,3,0,male,18.0,0,0
4,3,1,female,19.0,1,0
5,3,0,male,32.0,0,0
6,1,0,male,24.0,0,1
7,1,1,female,50.0,0,1
8,2,1,male,1.0,2,1
9,2,1,female,4.0,2,1


In [120]:
df_tidy = df_tidy[['pclass', 'sex', 'age', 'sibsp', 'parch', 'survived']]

#
# let's make sure it's all floating-point, so we can multiply and divide
#
A = A.astype('float64')  # so many types:  www.tutorialspoint.com/numpy/numpy_data_types.htm
print(A)


In [121]:
print("+++ Start of data definitions +++\n")

#
# we could do this at the data-frame level, too!
#

X_all = A[:, 0:-1] # X (features) all columns except 'survived'
y_all = A[:, -1]    # y (labels) ... is all rows, column 4 only

print(f"y_all (just the labels)   are \n {y_all}")
print(f"X_all (just the features) are \n {X_all}")

+++ Start of data definitions +++

y_all (just the labels)   are 
 [1. 1. 0. ... 0. 0. 0.]
X_all (just the features) are 
 [[ 0.      0.     29.      0.      0.    ]
 [ 0.      1.      0.9167  1.      2.    ]
 [ 0.      0.      2.      1.      2.    ]
 ...
 [ 0.      1.     26.5     0.      0.    ]
 [ 0.      1.     27.      0.      0.    ]
 [ 0.      1.     29.      0.      0.    ]]


In [122]:
#
# we scramble the data, to remove (potential) dependence on its ordering: 
# 
indices = np.random.permutation(len(y_all))  # indices is a permutation-list

# we scramble both X and y, necessarily with the same permutation
X_labeled = X_all[indices]              # we apply the _same_ permutation to each!
y_labeled = y_all[indices]              # again...
print(f"The scrambled labels/species are \n {y_labeled}")
print(f"The corresponding data rows are \n {X_labeled}")


The scrambled labels/species are 
 [0. 1. 0. ... 0. 1. 1.]
The corresponding data rows are 
 [[ 0.  1. 60.  0.  0.]
 [ 0.  0.  5.  0.  0.]
 [ 0.  1. 25.  0.  0.]
 ...
 [ 0.  1. 21.  0.  0.]
 [ 0.  1. 37.  1.  1.]
 [ 0.  0. 22.  0.  0.]]


In [123]:
#
# We next separate into test data and training data ... 
#    + We will train on the training data...
#    + We will _not_ look at the testing data to build the model
#
# Then, afterward, we will test on the testing data -- and see how well we do!
#

#
# a common convention:  train on 80%, test on 20%    Let's define the TEST_PERCENT
#

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2) # random_state=42

print(f"training with {len(y_train)} rows;  testing with {len(y_test)} rows\n" )

print(f"Held-out data... (testing data: {len(y_test)} rows)")
print(f"y_test: {y_test}")
print(f"X_test (first few rows): {X_test[0:5,:]}")  # 5 rows
print()
print(f"Data used for modeling... (training data: {len(y_train)} rows)")
print(f"y_train: {y_train}")
print(f"X_train (first few rows): {X_train[0:5,:]}")  # 5 rows

training with 803 rows;  testing with 201 rows

Held-out data... (testing data: 201 rows)
y_test: [1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0.
 1. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0.
 1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1.
 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1.
 1. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 0.
 0. 0. 0. 1. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 0. 1.
 1. 0. 0. 0. 0. 1. 1. 0. 0.]
X_test (first few rows): [[ 0.  0. 27.  0.  0.]
 [ 0.  1. 41.  0.  0.]
 [ 0.  0. 60.  1.  0.]
 [ 0.  1. 45.  1.  1.]
 [ 0.  0. 76.  1.  0.]]

Data used for modeling... (training data: 803 rows)
y_train: [0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0.
 1. 1. 0. 0. 

In [124]:
#
# Here, we use "cross validation" to find the "best" k...
#

from sklearn.model_selection import cross_val_score

#
# cross-validation splits the training set into two pieces:
#   + model-building and model-validation. We'll use "build" and "validate"
#
best_k = 84  # Not correct!
best_accuracy = 0.0  # also not correct...

# Note that we are cross-validating using only our TEST data!
for k in range(1,42):
    knn_cv_model = KNeighborsClassifier(n_neighbors=k)   # build a knn_model for every k
    cv_scores = cross_val_score( knn_cv_model, X_train, y_train, cv=5 )  # cv=5 means 80/20
    this_cv_accuracy = cv_scores.mean()               # mean() is numpy's built-in average function 
    print(f"k: {k:2d}  cv accuracy: {this_cv_accuracy:7.4f}")

    if this_cv_accuracy > best_accuracy:  
        best_accuracy = this_cv_accuracy  # track the best accuracy
        best_k = k                        # with the best k

    
# use best_k!
print(f"best_k = {best_k}   yields the highest average cv accuracy.")  # print the best one


k:  1  cv accuracy:  0.6738
k:  2  cv accuracy:  0.6875
k:  3  cv accuracy:  0.6899
k:  4  cv accuracy:  0.6837
k:  5  cv accuracy:  0.6899
k:  6  cv accuracy:  0.6912
k:  7  cv accuracy:  0.6974
k:  8  cv accuracy:  0.7037
k:  9  cv accuracy:  0.6912
k: 10  cv accuracy:  0.6875
k: 11  cv accuracy:  0.6937
k: 12  cv accuracy:  0.6962
k: 13  cv accuracy:  0.6949
k: 14  cv accuracy:  0.6974
k: 15  cv accuracy:  0.7062
k: 16  cv accuracy:  0.6875
k: 17  cv accuracy:  0.6974
k: 18  cv accuracy:  0.6800
k: 19  cv accuracy:  0.6813
k: 20  cv accuracy:  0.6763
k: 21  cv accuracy:  0.6838
k: 22  cv accuracy:  0.6676
k: 23  cv accuracy:  0.6825
k: 24  cv accuracy:  0.6613
k: 25  cv accuracy:  0.6613
k: 26  cv accuracy:  0.6588
k: 27  cv accuracy:  0.6675
k: 28  cv accuracy:  0.6613
k: 29  cv accuracy:  0.6501
k: 30  cv accuracy:  0.6463
k: 31  cv accuracy:  0.6526
k: 32  cv accuracy:  0.6351
k: 33  cv accuracy:  0.6352
k: 34  cv accuracy:  0.6339
k: 35  cv accuracy:  0.6302
k: 36  cv accuracy: 

In [125]:
#
# With the best k, we build and train a new model:
#
# Now using best_k instead of the original, randomly-guessed value:   
#
best_k = best_k   # not needed, but nice
from sklearn.neighbors import KNeighborsClassifier
knn_model_tuned = KNeighborsClassifier(n_neighbors=best_k)   # here, we use the best_k!

# we train the model (one line!)
knn_model_tuned.fit(X_train, y_train)                              # yay!  trained!
print(f"Created + trained a knn classifier, now tuned with a (best) k of {best_k}")  

# How does it do?!  The next cell will show...

Created + trained a knn classifier, now tuned with a (best) k of 15


In [126]:
#
# Re-create and re-run the  "Model-testing Cell"     How does it do with best_k?!
#
predicted_labels = knn_model_tuned.predict(X_test)
actual_labels = y_test

# Let's print them so we can compare...
print("Predicted labels:", predicted_labels)
print("Actual labels:", actual_labels)

# And, the overall results
num_correct = sum(predicted_labels == actual_labels)
total = len(actual_labels)
print(f"\nResults on test set:  {num_correct} correct out of {total} total.\n\n")

# Plus, we'll print our nicer table...
compare_labels(predicted_labels,actual_labels)

Predicted labels: [1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0.
 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 1. 0.
 0. 0. 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 0. 0.]
Actual labels: [1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0.
 1. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0.
 1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1.
 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 0

135

##### The best k for these 42 additional titanic passengers is 15 with a cv accuracy of 0.7062 and the testing accuracy comes out to be 135/201. 

### The best cross validation score I got over this entire program is 0.7759 with a best k of 7. This is for the test where 'sex' was heavily more considered over every other feature. The model was able to more accurately predict the survival of the titanic passengers when gender is heavily more considered than any other feature. 

### Personally, I believe age is the most important feature as explained in the previous tests. Those who are too young or too old cannot care for themselves in such a tragic event. I believe middle-aged adults have the highest chances of survival. With this being said, all of these features may not even matter and the entire survival rate in the Titanic could be all luck. 